# IPHOTO ALBUM (& NAS) DOWNLOAD TOOL & KML PARSER

#### This tool downloads a local copy of a specified and authenticated iCloud photo album or NAS storage. After generating the local duplicate, the tool scans each image and compiles an inventory, incorporating GPS data. The inventory, complete with GPS coordinates, is saved as a CSV file. This file is then be parsed to identify photos taken within locations or projects defined by KML polygons exported from Google Earth. The tool then copies the identified photos into project-specific folders.

## STEP 1: START HERE TO LOAD MODULES AND BUILD SUB-ROUTINES

In [1]:
from datetime import datetime
from exif import Image
import os
import io
import pyicloud
from pyicloud import PyiCloudService
import geopandas as gpd
import time
import csv
from PIL import Image as PILImage
from pillow_heif import HeifImagePlugin
from PIL import ExifTags
import hashlib
import shutil
import fiona
from shapely.geometry import Point, Polygon
import pandas as pd
from csv import reader
fiona.drvsupport.supported_drivers['KML'] = 'rw'

def icloud_initiate(username = "", password = ""):
    # Authenticate with your iCloud account
    api = PyiCloudService(username, password)
    if api.requires_2fa:
        print("Two-factor authentication required.")
        code = input("Enter the code you received of one of your approved devices: ")
        result = api.validate_2fa_code(code)
        print("Code validation result: %s" % result)
        if not result:
            print("Failed to verify security code")
            sys.exit(1)
        if not api.is_trusted_session:
            print("Session is not trusted. Requesting trust...")
            result = api.trust_session()
            print("Session trust result %s" % result)
            if not result:
                print("Failed to request trust. You will likely be prompted for the code again in the coming weeks")
    elif api.requires_2sa:
        import click
        print("Two-step authentication required. Your trusted devices are:")
        devices = api.trusted_devices
        for i, device in enumerate(devices):
            print(
                "  %s: %s" % (i, device.get('deviceName',
                "SMS to %s" % device.get('phoneNumber')))
            )
        device = click.prompt('Which device would you like to use?', default=0)
        device = devices[device]
        if not api.send_verification_code(device):
            print("Failed to send verification code")
            sys.exit(1)
        code = click.prompt('Please enter validation code')
        if not api.validate_verification_code(device, code):
            print("Failed to verify verification code")
            sys.exit(1)
    return(api)

def download_missing_icloud_photos(destination_folder = "", api=""):
    tic = time.perf_counter()
    photos = api.photos.all
    files_checked_count = 0
    new_files_found_count = 0
    files_downloaded_list = []
    total_count = len(photos)
    local_file_list=[]
    for file in os.listdir(destination_folder):
        local_file_list.append(os.path.splitext(file)[0])
    for photo in photos:
        files_checked_count +=1
        file_extension = os.path.splitext(photo.filename)
        new_filename = format(photo.created.timestamp(),".3f") + file_extension[1]
        photo_filename = os.path.join(destination_folder, new_filename)
        if os.path.splitext(new_filename)[0] in local_file_list:
            pass
        else:
            new_files_found_count +=1
            image_data = photo.download().content
            try:
                img = PILImage.open(io.BytesIO(image_data))
                image_exif = img.getexif()
                image_exif[0xA300] = "iCloud"
                img.save(photo_filename, exif= image_exif)
            except:
                with open(photo_filename, 'wb') as opened_file:
                    opened_file.write(image_data)
            files_downloaded_list.append(photo_filename)
        print("New iPhotos found on icloud = " + str(new_files_found_count) + " (Checking: " + str(files_checked_count) + " of " + str(total_count) + ")", end ="\r")
    print("\nMissing photos download done! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return(files_downloaded_list)

def download_new_icloud_photos(destination_folder = "", api=""):
    photos = api.photos.all
    files_checked_count = 0
    new_files_found_count = 0
    existing_files_found_count = 0
    files_downloaded_list = []
    total_count = len(photos)
    local_file_list=[]
    for file in os.listdir(destination_folder):
        local_file_list.append(os.path.splitext(file)[0])
    for photo in photos:
        files_checked_count +=1
        file_extension = os.path.splitext(photo.filename)
        new_filename = format(photo.created.timestamp(),".3f") + file_extension[1]
        photo_filename = os.path.join(destination_folder, new_filename)
        if os.path.splitext(new_filename)[0] in local_file_list:
            existing_files_found_count += 1
            pass
        else:
            new_files_found_count +=1
            existing_files_found_count = 0
            image_data = photo.download().content
            try:
                img = PILImage.open(io.BytesIO(image_data))
                image_exif = img.getexif()
                image_exif[0xA300] = "iCloud"
                img.save(photo_filename, exif= image_exif)
            except:
                with open(photo_filename, 'wb') as opened_file:
                    opened_file.write(image_data)
            files_downloaded_list.append(photo_filename)
        print("New iPhotos found on icloud = " + str(new_files_found_count) + " (Checking: " + str(files_checked_count) + " of " + str(total_count) + ")", end ="\r")
        if existing_files_found_count > 1000:
            break
    print("\nNew photos download done! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    return(files_downloaded_list)

def download_all_icloud_photos_again(destination_folder = "", api=""):
    tic = time.perf_counter()
    exist_file_count = len(os.listdir(destination_folder))
    photos = api.photos.all
    files_downloaded_count = 0
    files_downloaded_list =[]
    total_count = len(photos)
    for photo in photos:
        files_downloaded_count +=1
        if files_downloaded_count < (exist_file_count - 5):
            print("iPhoto already downloaded count = " + str(files_downloaded_count), end ="\r")
            continue
        file_extension = os.path.splitext(photo.filename)
        new_filename = format(photo.created.timestamp(),".3f") + file_extension[1]
        photo_filename = os.path.join(destination_folder, new_filename)
        image_data = photo.download().content
        try:
            img = PILImage.open(io.BytesIO(image_data))
            image_exif = img.getexif()
            image_exif[0xA300] = "iCloud"
            img.save(photo_filename, exif= image_exif)
        except:
            with open(photo_filename, 'wb') as opened_file:
                opened_file.write(image_data)
        files_downloaded_list.append(photo_filename)
        print("iPhotos downloaded from icloud = " + str(files_downloaded_count) + " of " + str(total_count), end ="\r")
    print("\niCloud photos full download complete! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def quick_photo_csv(target_folders = []):
    print("Running quick_photo_csv")
    tic = time.perf_counter()
    photo_list_file = os.path.join(system_file_location, "GPS_image_list_master.csv")
    file_list = []
    file_list_name = []
    if os.path.exists(photo_list_file) != True:
        file_list=[["File Name", "File type", "Source location", "Date", "Lattitude", "Longitude"]]
    else:
        with open(photo_list_file, 'r') as file:
            csvreader = csv.reader(file)
            for row in csvreader:
                file_list.append(row)
                file_list_name.append(row[0])
    print("Opened 'GPS_image_list_master.csv'. Looking for missed records.")
    files_checked_count = 0
    new_files_found = 0
    all_filenames_found = []
    for folder in target_folders:
        for file in get_all_filenames_in_target_folder(folder):
            all_filenames_found.append(file)
    photo_quantity = len(all_filenames_found)
    for photo in all_filenames_found:
        files_checked_count +=1
        if os.path.splitext(os.path.basename(photo))[0] in file_list_name:
            pass
        else:
            if os.path.splitext(photo)[1] == ".JPEG" or os.path.splitext(photo)[1] == ".MOV":
                new_files_found +=1
                image_exif = get_image_coordinates(photo)
                date = image_exif[0]
                coords_lat = image_exif[1]
                coords_long = image_exif[2]
                file_source = photo
                full_path = photo
                photo = os.path.basename(photo)
                new_file=[os.path.splitext(photo)[0],os.path.splitext(photo)[1],file_source, date, coords_lat, coords_long]
                file_list.append(new_file)
        print("Checking file " + str(files_checked_count) + " of " + str(photo_quantity) + ". New JPG or MOV files found = " + str(new_files_found), end ="\r")
    with open(photo_list_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(file_list)
    print("\nUpdated photo register written to 'GPS_image_list_master.csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def full_photo_csv(target_folders = []):
    tic = time.perf_counter()
    print("Proceeding to gather info on disk contents. This will take a while.")
    file_list=[["File Name", "File type", "Source location", "Date", "Lattitude", "Longitude"]]
    files_checked_count = 0
    all_filenames_found = []
    for folder in target_folders:
        for file in get_all_filenames_in_target_folder(folder):
            all_filenames_found.append(file)
    photo_quantity = len(all_filenames_found)
    for photo in all_filenames_found:
        files_checked_count +=1
        if os.path.splitext(photo)[1] == ".JPEG" or os.path.splitext(photo)[1] == ".MOV":
            image_exif = get_image_coordinates(photo)
            date = image_exif[0]
            coords_lat = image_exif[1]
            coords_long = image_exif[2]
            file_source = photo
            full_path = photo
            photo = os.path.basename(photo)
            new_file=[os.path.splitext(photo)[0],os.path.splitext(photo)[1],file_source, date, coords_lat, coords_long]
            file_list.append(new_file)
        print("Photos resolved = " + str(files_checked_count) + " of " + str(photo_quantity), end ="\r")
    photo_list_file = os.path.join(system_file_location, "GPS_image_list_master.csv")
    with open(photo_list_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(file_list)
    print("\nNew photo register written to 'GPS_image_list_master.csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def get_image_coordinates(image_file):
    date = ''
    coords_lat = 0
    coords_long = 0
    file_source = ""
    try:
        with open(image_file, 'rb') as src:
            img = Image(src)
            date = ''
            coords = (0,0)
            if img.has_exif:
                try:
                    pass
                    imge = PILImage.open(image_file)
                    image_exif = imge.getexif()
                    file_source = image_exif[0xA300]
                except:
                    pass
                try:
                    date = img.datetime_original
                except:
                    pass
                try:
                    coords_lat = decimal_coords(img.gps_latitude,img.gps_latitude_ref)
                    coords_long = decimal_coords(img.gps_longitude,img.gps_longitude_ref)
                except:
                    pass
            else:
                pass
    except:
        if os.path.splitext(image_file)[1] != ".MOV":
            print("File may be corrupt: " + image_file)
        pass
    return (date, coords_lat, coords_long, file_source)

def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == 'S' or ref == 'W':
        decimal_degrees = -decimal_degrees
    return decimal_degrees

def convert_images_to_JPEG(target_folder = ""):
    print("Running convert_images_to_JPEG")
    succesful_conversion = 0 
    failed_conversion = 0 
    files_to_convert=[]
    files_not_converted = []
    list_of_files = get_all_filenames_in_target_folder(target_folder)
    print("Analysing folder")
    for file in list_of_files:
        file_extension = os.path.splitext(file)[1]
        if file_extension != ".JPEG" and file_extension != ".MOV":
            files_to_convert.append(file)
    full_count = len(files_to_convert)
    for file_with_path in files_to_convert:
        print("Files converted to JPEG: " + str(succesful_conversion) + " of " + str(full_count), end ="\r")
        if os.path.exists(file_with_path):
            file_extension = os.path.splitext(file_with_path)[1]
            if file_extension != ".JPEG" and file_extension != ".MOV":
                new_filename = os.path.splitext(file_with_path)[0] + ".JPEG"
                try:
                    #print(path)
                    with PILImage.open(file_with_path) as img:
                        image_exif = img.getexif()
                        img.format = 'jpeg'
                        img.save(new_filename, exif= image_exif, quality=75)
                        succesful_conversion +=1
                    os.remove(file_with_path)
                except:
                    failed_conversion += 1
                    files_not_converted.append(file_with_path)
    if len(files_not_converted) != 0:
        print("\nFiles that were not converted = " + str(failed_conversion) + ", moved to " + unsupported_file_location + ": ")
        for file in files_not_converted:
            try:
                os.rename(file, os.path.join(unsupported_file_location, os.path.basename(file)))
            except:
                os.remove(os.path.join(unsupported_file_location, os.path.basename(file)))
            print(os.path.basename(file))
    return()

def get_all_filenames_in_target_folder(target_folder = ""):
    dir_path = target_folder
    list_of_files = []
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            list_of_files.append(os.path.join(dir_path, path))
    print("Files found on " + target_folder + ": " + str(len(list_of_files)))
    return(list_of_files)

def get_hash(f_path, mode='md5'):
    h = hashlib.new(mode)
    with open(f_path, 'rb') as file:
        data = file.read()
    h.update(data)
    digest = h.hexdigest()
    return digest

def crawl_for_images(destination_folder = "", source_locations = []):
    tic = time.perf_counter()
    files_downloaded_list = []
    failed_to_download_list = []
    file_list = []
    files_downloaded_count = 0
    total_count = 0
    files_to_check_for = (".HEIC", ".heic",".jpeg", ".JPEG", ".JPG", ".jpg")
    for folder_location in source_locations:
        file_list = []
        for root, dirs, files in os.walk(folder_location, topdown=False):
            print("                                                                                                                                                                                                                                                                                         ", end ="\r")
            print("Folder being checked = " + root, end ="\r")
            for filename in files:
                if filename.lower().endswith(files_to_check_for) == True:
                    full_file_path = os.path.join(root, filename)
                    if os.path.getsize(full_file_path) > smallest_image_size:
                        file_list.append(full_file_path)
        total_count = len(file_list) + total_count
        print("\n")
        for photo in file_list:
            files_downloaded_count += 1
            file_extension = ".JPEG"
            photo_exif = get_image_coordinates(photo)
            date_str = photo_exif[0]
            if date_str != '':
                try:
                    date_format = '%Y:%m:%d %H:%M:%S'
                    date_obj = datetime.strptime(date_str, date_format)
                    new_filename = format(datetime.timestamp(date_obj),".3f") + file_extension
                except:
                    try:
                        new_filename = "CRC." + get_hash(photo) + file_extension  
                    except:
                        #Handles over length paths
                        failed_to_download_list.append(photo_filename)
                        pass
            else:
                try:
                    new_filename = "CRC." + get_hash(photo) + file_extension
                except:
                    #Handles over length paths
                    failed_to_download_list.append(photo_filename)
                    pass
            if photo_exif[1] != 0 and photo_exif[2] != 0:
                photo_filename = os.path.join(destination_folder, new_filename)
                try:
                    img = PILImage.open(photo)
                    image_exif = img.getexif()
                    image_exif.update({41728: photo})
                    img.format = 'jpeg'
                    if not os.path.exists(photo_filename):
                        img.save(photo_filename, exif= image_exif, quality=75)
                    files_downloaded_list.append(photo_filename)
                except:
                    failed_to_download_list.append(photo_filename)
                    print("Problem saving to drive = " + photo_filename, end ="\r")
                    pass
            print("Crawled Photos downloaded = " + str(files_downloaded_count) + " of " + str(total_count) + "                                   ", end ="\r")
    print("\nCrawled photos full download complete! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def copy_found_images(images_found, path):
    for images in images_found:
        source = images[1]
        #print("images =" + source)
        #print("path=" + path)
        destination = path +  "\\" + images[0] + "\\" + os.path.basename(source)
        if not os.path.exists(destination):
            os.makedirs(os.path.dirname(destination), exist_ok=True)
            try:
                #print(source, destination)
                shutil.copy(source, destination)
                print('Image copied succesfully to: ' + destination +"                                                                         " , end ="\r")
            except shutil.SameFileError:
                attempt=0
                while attempt <= 10:
                    attempt +=1
                    destination=destination+'-(same_name_'+str(attempt)+')'
                print("Source and destination represents the same file. Not copied: ", source)
            except:
                print("Error occurred while copying file. Not copied: ", source)  
    print("Copy found_images - Done!")
  
def write_list_to_disk(raw_list, paths, file_name_prefix):
    for path in paths:
        if os.path.exists(path):
            sys_timestamp=time.time()
            file_name="%s-%s.csv" % (file_name_prefix, sys_timestamp)
            csv_file_name=os.path.join(path, file_name)
            with open(csv_file_name,"w", newline='') as csv_file:
                write = csv.writer(csv_file)
                write.writerows(raw_list)
            print("Image list sucessfully written to disk at %s" % csv_file_name)
        else:
            print("Path not accessible for writing" + path)

def write_master_to_disk(raw_list, paths, file_name):
    for path in paths:
        if os.path.exists(path):
            sys_timestamp=time.time()
            file_name="%s.csv" % (file_name)
            csv_file_name=os.path.join(path, file_name)
            with open(csv_file_name,"w", newline='') as csv_file:
                write = csv.writer(csv_file)
                write.writerows(raw_list)
            print("Image list sucessfully written to disk at %s" % csv_file)
        else:
            print("Path not accessible for writing" + path)
            
def find_any_image_files(root_paths):
    file_list=[]
    for root_path in root_paths:
        if os.path.exists(root_path):
            print("Root path is found: " + root_path)
            print('Busy looking for images')
            for (root,dirs,files) in os.walk(root_path, topdown=1): 
                for file in files:
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        file_found_with_path=os.path.join(root, file)
                        file_list.append([file_found_with_path])
            print('Done')
        else:
            print("Root path is not found. Please check path accessibility: " + root_path)
    return(file_list)

def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == 'S' or ref == 'W':
        decimal_degrees = -decimal_degrees
    return decimal_degrees

def image_coordinates(image_list):
    GPS_list=[]
    for img_path in image_list:
        image_file=img_path[0]
        print(image_file)
        fail = 0
        try:
            with open(image_file, 'rb') as src:
                img = Image(src)
                date=('')
                coords=(0,0)
                if img.has_exif:
                    try:
                        date = img.datetime_original
                    except:
                        pass
                    try:
                        coords_lat = decimal_coords(img.gps_latitude,img.gps_latitude_ref)
                        coords_long = decimal_coords(img.gps_longitude,img.gps_longitude_ref)
                    except AttributeError:
                        print ('No Coordinates')
                        fail=1
                    if fail != 1:
                        GPS_list.append([src.name, date, coords_long, coords_lat])
                        print(f"Image {src.name}, Was taken: {img.datetime_original}, and has coordinates: {coords_lat}, {coords_long}")
                else:
                    print ('The Image has no EXIF information')
        except:
            print('Image not valid '+ image_file)
    return GPS_list

def find_images_in_kml(kml_file, csv_file):
    print("Running find_images_in_kml")
    tic = time.perf_counter()
    my_sites = gpd.list_layers(kml_file) 
    image_found_list=[]
    for layer in my_sites.name:
        my_map = gpd.read_file(kml_file, driver='KML', layer = layer)
        #print(my_map)
        last_found_photo = 0
        for i in range(0, len(my_map)):
            x=0
            folder = os.path.join(layer, my_map.Name.loc[i])
            #print(folder)
            # open file in read mode
            with open(csv_file, 'r') as read_obj:
                # pass the file object to reader() to get the reader object
                csv_reader = reader(read_obj)
                # Iterate over each row in the csv using reader object
                my_pass=0
                last_found_photo = 0
                for row in csv_reader:
                    if my_pass > 0:
                        # row variable is a list that represents a row in csv
                        p1=Point(float(row[5]),float(row[4]))
                        check = my_map.contains(p1) #.values
                        #print(check)
                        if check[i]==True:
                            datetime_taken = row[3]
                            if datetime_taken == "":
                                datetime_taken = "1971:01:01 00:00:00"
                            date_time_obj=datetime.strptime(datetime_taken, '%Y:%m:%d %H:%M:%S')
                            date_str = date_time_obj.strftime("%Y-%m-%d")
                            directory = os.path.join(folder,date_str)
                            #print(directory)
                            print(directory + "                                        ", end ="\r")
                            image_found_list.append([directory,row[2]]) 
                            last_found_photo = date_time_obj.timestamp()
                        else:
                            datetime_taken = row[3]
                            if datetime_taken != "":
                                date_time_obj=datetime.strptime(datetime_taken, '%Y:%m:%d %H:%M:%S')
                                if abs(last_found_photo - date_time_obj.timestamp()) < 300:
                                    date_str = date_time_obj.strftime("%Y-%m-%d")
                                    directory = os.path.join(folder,date_str)
                                    print(directory + "                                        ", end ="\r")
                                    #print(abs(last_found_photo - date_time_obj.timestamp()))
                                    image_found_list.append([directory,row[2]]) ##
                    my_pass = 1
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return(image_found_list)

## STEP 2: THEN SET PHOTO FOLDER LOCATION AND iCLOUD LOGIN CREDENTIALS

In [2]:
##Authentication credentials and setting photo location folders

#iCloud credentials
username = 'cdegouveia@rocketmail.com'
password = 'carisaCDG@3340'

#General settings
smallest_image_size = 250000

#Photo crawling locations
crawl_photo_sources = ["\\\\192.168.1.10\\File_cabinet\\Carlos"]

#Specify the KML file
kml_file='E:\\_system\\GPS Locations for Python.kml'

#Specify the CSV file containing the image list and the respective GPS coordinates.
#Make sure the CSV names corresponds with the file name specified during the master file creation process
csv_file='E:\\_system\\GPS_image_list_master.csv'

#Specify destination folder where the selected images will be copied to
destination_folder=('E:\\FOUND_IMAGES')

#Photo location folders
unsupported_file_location = "E:\\_unsupported_files"
system_file_location = "E:\\_system"
crawl_photo_destination = 'E:\\PHOTO_LIBRARY_CRAWL'
iphoto_folder_destination = 'E:\\PHOTO_LIBRARY_ICLOUD'

#Check folders exist
if not os.path.exists(unsupported_file_location):
    os.makedirs(unsupported_file_location)
if not os.path.exists(system_file_location):
    os.makedirs(system_file_location)
if not os.path.exists(crawl_photo_destination):
    os.makedirs(crawl_photo_destination)
if not os.path.exists(iphoto_folder_destination):
    os.makedirs(iphoto_folder_destination)

## STEP 3: COME HERE TO DOWNLOAD PHOTOS FROM iCLOUD

In [ ]:
#Download NEW photos from iPhoto album. Quickest!
#This can run in the background as the CSV file will not be updated automatically.
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_new_icloud_photos(iphoto_folder_destination, api)
convert_images_to_JPEG(iphoto_folder_destination)

In [ ]:
#Download MISSING photos from iPhoto album. Not the quickest. This will a little time to complete!
#This can run in the background as the CSV file will not be updated automatically.
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_missing_icloud_photos(iphoto_folder_destination, api)
convert_images_to_JPEG(iphoto_folder_destination)

In [ ]:
#Download FULL content of iPhoto album. This is SLOW!!! This will take many hours to complete!
#This can run in the background as the CSV file will not be updated automatically.
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_all_icloud_photos_again(iphoto_folder_destination, api)
convert_images_to_JPEG(iphoto_folder_destination)

## STEP 4: (OPTIONAL) COME HERE TO CRAWL FOR PHOTOS FROM NAS

In [ ]:
crawl_for_images(destination_folder = crawl_photo_destination, source_locations = crawl_photo_sources)
convert_images_to_JPEG(crawl_photo_destination)

## STEP 5: PHOTO CONVERSION

In [ ]:
# Convert iPhoto to JPEG
convert_images_to_JPEG(iphoto_folder_destination)

In [ ]:
# Convert Blackbox to JPEG
convert_images_to_JPEG(crawl_photo_destination)

## STEP 6: COME HERE TO UPDATE .CSV PHOTO REGISTER

In [ ]:
#Check the photo register for missing photos. Quickest!
quick_photo_csv([iphoto_folder_destination, crawl_photo_destination])

In [ ]:
#Do a full refresh of the photo register. Not the quickest. This will take a couple hours!
full_photo_csv([iphoto_folder_destination, crawl_photo_destination])

## STEP 7: USE THIS TO PARSE THE KML FILE AND FIND THE MATCHING IMAGES IN THE CSV AND THEN COPY THE IMAGES TO THE DESTINATION

In [ ]:
#Use this to locate images within the KML polygons.
#The images will be collected from the source destinations (make sure
#the destination folders are accessible) and copied to the destination folder.

#Sift through the list of GPS coordinates. If an image falls within a KML
#polygon, then put it into a list.
qualifying_image_list=find_images_in_kml(kml_file, csv_file)

#Take the list and use it to copy the qualifying images to their respective folders.
#The folders names coorespond with the polygon names created with GOOGLE Earth.
#The KML file can hold several polygons. Each polygon must have a unique name.
copy_found_images(qualifying_image_list, destination_folder)

## OR STEP 3: COME HERE TO PERFORM A DAILY TYPICAL QUICK 'NEW iPHOTO'S DOWNLOAD, A QUICK REGISTER UPDATE, A KML PARSE AND FINALLY A PHOTOS FOUND DISK COPY'.

In [3]:
#Use this for day to day use
tic = time.perf_counter() #Start Time taken timer
api = icloud_initiate(username = username, password = password)                        #Login to iCloud
download_new_icloud_photos(iphoto_folder_destination, api)                             #Download photos
convert_images_to_JPEG(iphoto_folder_destination)                                      #Convert downloaded photos to my standard format
quick_photo_csv([iphoto_folder_destination])                                           #Update CSV file with new photos and GPS
qualifying_image_list=find_images_in_kml(kml_file, csv_file)                           #Parse the CSV to find photos that fall in the KML polygons
destination_folder=('E:\\FOUND_IMAGES')                                                #Copy found photos to USB
copy_found_images(qualifying_image_list, destination_folder)
destination_folder=('D:\\OneDrive - Department of Premier Western Cape\\Site Photos')  #Copy found photos to onedrive
copy_found_images(qualifying_image_list, destination_folder)
print(f"TOTAL Time taken: {(time.perf_counter() - tic)/60:0.4f} minutes") #Time taken result

New iPhotos found on icloud = 0 (Checking: 501 of 35367)
New photos download done! Now run 'quick_photo_csv' or 'full_photo_csv'.
Running convert_images_to_JPEG
Files found on E:\PHOTO_LIBRARY_ICLOUD: 35345
Analysing folder
Files converted to JPEG: 0 of 5
Files that were not converted = 5, moved to E:\_unsupported_files: 
1651403582.816.PNG
1606983960.000.mp4
1525354390.000.png
1512065435.044.PNG
1512065201.636.PNG
Running quick_photo_csv
Opened 'GPS_image_list_master.csv'. Looking for missed records.
Files found on E:\PHOTO_LIBRARY_ICLOUD: 35340
Checking file 35340 of 35340. New JPG or MOV files found = 0
Updated photo register written to 'GPS_image_list_master.csv'.
Time taken: 3.5449 minutes
Running find_images_in_kml
Time taken: 4.8215 minutes4-07-11                                                                          
Copy found_images - Done!
Copy found_images - Done!
TOTAL Time taken: 11.9098 minutes


## NOTES FOR FUTURE IMPROVEMENT
- remove requirement for module EXIF
- Save blackbox photos to seperate directories named after the crawl sources
- have a csv feature that looks in the directory source folder of the photo and check if there is a kml in order to fill missing GPS coordinates
- Introduce a feature that removes obsolete photos (that were deleted from iphone or blackbox)
- When doing a CSV registration, maybe instead of specifying the folders it needs to check, instead do a crawl of the drive looking for JPEG images, and ignore the _unsupported_files folder
- When downloading iCloud, and checking if files exists before saving, maybe check _unsupported_files folder as well (not just the main folder). Preventing a save of an existing image will prevent overwriting any new exif data this app wrote to the image
- Introduce a 'Learn' feature where the images are updated from changes to the CSV.
- Intrduce CSV feature that automatically populates GPS coordinates according to likely co-ordinates based on neighbouring images. Tthen apply the 'Learn' feature. This way the original images don't need to be modified.
- When a CSV is updated, save the previous CSV as a revision so that errors can be rolled back
- Someteims when the app runs, warnings appear. See if these can be handled.
- The Full CSV does not pick up MOV files, while the quick CSV picks them up. I think the Full CSV should also pick up MOV in order to get them approximate GPS coordinates
- MOV files have 'media created' exif. Does python see this as 'date taken'? It seems like it does, but maybe it is the EXIF module that does this automatically and might not work if PIL EXIF is used because it might be a different tag
- Consider combining all photos (blackbox and ios) into one folder so that duplicates are not saved
- Create a feature that creates kml and popuplates these pins on to google earth, maybe including photos. Maybe moving the pins to use the learn feature to modify the CSV and the affected photo in the usb drive.
- Checking whether a file is already prsent in the drive or in the CSV list can be improved using a binomial sort alglorith. Taking too long to perform 'quick csv check'
- In crawl algorithm, the chçheck if file in csv exsts' might not be working
- The subroutines must test for presence of drive. All the try's trigger the print "Corrucpt...." line  
- Binomial search algorith. Apply this to all search algorithms, including the GPS search algorith.
- Something wrong with quick CSV. Didn't pick up new 1000's additions from crawl.
- For crawl, maybe argument should be a single target folder at a time instead of a list of folders.
- For population of missing GPS coords, rather copy the missing coords to the CSV and then use learn feature to send it back to the usb drives images